<a href="https://colab.research.google.com/github/tawanamutasa/Weather-wise-Tawana-mutasa-/blob/main/Weatherwise_Tawana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Weather Wise App **

In [5]:
import requests
import matplotlib.pyplot as plt
from datetime import datetime
from geopy.geocoders import Nominatim

In [6]:
def get_coordinates(city_name):
    geolocator = Nominatim(user_agent="weather_app")
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    else:
        raise ValueError("Location not found.")

In [11]:
def get_weather_forecast(lat, lon):
    url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={lat}&longitude={lon}"
        f"&hourly=temperature_2m,precipitation_probability"
        f"&timezone=auto"
    )
    response = requests.get(url)
    return response.json()